In [1]:
import pandas as pd
import surprise
from surprise.prediction_algorithms import SVD, SVDpp, SlopeOne, NMF, NormalPredictor, KNNBaseline, KNNBasic, KNNWithMeans, KNNWithMeans, KNNWithZScore, BaselineOnly, CoClustering 
KNNWithMeans, KNNWithZScore, BaselineOnly, CoClustering
from surprise.model_selection.validation import cross_validate
from surprise.model_selection.split import ShuffleSplit

In [2]:
filepath = '/Users/talhamchaudhry/Desktop/machine_learning/project/data/data-full.txt'

In [15]:
ratings = pd.read_csv(filepath, header = None, names =['MovieID', 'UserID', 'Rating'])

In [16]:
dirty = ratings.groupby('UserID')['Rating'].count()

In [53]:
len(ratings.loc[ratings.MovieID == 15582])

20873

In [30]:
A = ratings.groupby('UserID')[['Rating']].count().sort_index().sort_values('Rating', ascending = False)

In [65]:
A.std()

Rating    3.592898
dtype: float64

In [64]:
len(A.loc[A.Rating > 6]), len(A), len(A.loc[A.Rating < 6]), len(A.loc[A.Rating == 6])

(18007, 28968, 6922, 4039)

In [48]:
B = ratings.groupby('MovieID')[['Rating']].count().sort_index().sort_values('Rating', ascending = False)

In [51]:
B.head()

,Rating
MovieID,
15582,20873
1744,20140
14712,19935
6482,19580
16082,16542


In [55]:
len(B.loc[B.Rating > 50]), len(B), len(B.loc[B.Rating < 50]), len(B.loc[B.Rating == 50])

(79, 92, 12, 1)

In [67]:
min_movie_ratings = 50
filter_movies = ratings['MovieID'].value_counts() > min_movie_ratings
filter_movies = filter_movies[filter_movies].index.tolist()

min_user_ratings = 6
filter_users = ratings['UserID'].value_counts() > min_user_ratings
filter_users = filter_users[filter_users].index.tolist()

new_data = ratings[(ratings['MovieID'].isin(filter_movies)) & (ratings['UserID'].isin(filter_users))]


In [69]:
ratings.shape, new_data.shape

((227773, 3), (173953, 3))

In [70]:
reader = surprise.Reader(rating_scale = (1,5))

data = surprise.Dataset.load_from_df(new_data[['UserID', 'MovieID', 'Rating']], reader)

In [13]:
def read_data(filepath):
    reader = surprise.Reader(line_format='item user rating', sep=',')
    data = surprise.Dataset.load_from_file(filepath, reader=reader)
    return data

In [14]:
data = read_data(filepath)

In [71]:
benchmark = []
for algorithm in [SVD(), SVDpp(), SlopeOne(), NMF(), NormalPredictor(), KNNBaseline(), KNNBasic(), KNNWithMeans(),
                 KNNWithZScore(), BaselineOnly(), CoClustering()]:
    results = cross_validate(algorithm, data, measures = ['RMSE'], cv = 3, verbose = False)
    
    tmp = pd.DataFrame.from_dict(results).mean(axis = 0)
    tmp = tmp.append(pd.Series([str(algorithm).split(' ')[0].split('.')[-1]], index = ['Algorithm']))
    benchmark.append(tmp)

pd.DataFrame(benchmark).set_index('Algorithm').sort_values('test_rmse')

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...


,test_rmse,fit_time,test_time
Algorithm,,,
SVDpp,0.927620,29.267755,1.721748
KNNBaseline,0.948147,430.403608,1682.759363
SVD,0.951378,8.832067,0.559029
BaselineOnly,0.957022,0.471925,0.617264
SlopeOne,0.964403,0.344215,1.002912
KNNWithMeans,0.969850,1884.102244,2437.623297
KNNBasic,0.976141,1910.221437,1156.325238
KNNWithZScore,0.976633,366.580873,2356.710875
CoClustering,1.023488,5.279346,0.501341


In [54]:
import chardet    
rawdata = open(filepath2, 'rb').read()
result = chardet.detect(rawdata)
charenc = result['encoding']

NameError: name 'filepath2' is not defined

In [37]:
charenc

'ISO-8859-1'

In [38]:
movies = pd.read_csv(filepath2, header = None, names =['MovieID', 'YearOfRelease', 'Title'], index_col=False, encoding = charenc)

In [45]:
movies.sample(4)

,MovieID,YearOfRelease,Title
10084,10085,2002.0,Zus & Zo
6766,6767,1988.0,Dead Heat
15990,15991,1997.0,Wilde
2796,2797,2003.0,Life of Buddha


In [46]:
def check_null(data):
    return data.isnull().sum()

In [47]:
check_null(movies)

MovieID          0
YearOfRelease    7
Title            0
dtype: int64

In [40]:
cond = movies.YearOfRelease.isnull()

In [49]:
movies.loc[cond]

,MovieID,YearOfRelease,Title
4387,4388,NaN,Ancient Civilizations: Rome and Pompeii
4793,4794,NaN,Ancient Civilizations: Land of the Pharaohs
7240,7241,NaN,Ancient Civilizations: Athens and Greece
10781,10782,NaN,Roti Kapada Aur Makaan
15917,15918,NaN,Hote Hote Pyaar Ho Gaya
16677,16678,NaN,Jimmy Hollywood
17666,17667,NaN,Eros Dance Dhamaka


In [50]:
movies.loc[cond]['YearOfRelease']

4387    NaN
4793    NaN
7240    NaN
10781   NaN
15917   NaN
16677   NaN
17666   NaN
Name: YearOfRelease, dtype: float64

In [41]:
names = movies.loc[cond].Title.tolist()

In [52]:
type(names)

list

In [53]:
names

['Ancient Civilizations: Rome and Pompeii',
 'Ancient Civilizations: Land of the Pharaohs',
 'Ancient Civilizations: Athens and Greece',
 'Roti Kapada Aur Makaan',
 'Hote Hote Pyaar Ho Gaya',
 'Jimmy Hollywood',
 'Eros Dance Dhamaka']

In [42]:
year = [2001, 2001, 2001, 1974, 1999, 1994, 1999]

In [43]:
my_dict = {n:y for n,y in zip(names, year)}

In [44]:
my_dict

{'Ancient Civilizations: Rome and Pompeii': 2001,
 'Ancient Civilizations: Land of the Pharaohs': 2001,
 'Ancient Civilizations: Athens and Greece': 2001,
 'Roti Kapada Aur Makaan': 1974,
 'Hote Hote Pyaar Ho Gaya': 1999,
 'Jimmy Hollywood': 1994,
 'Eros Dance Dhamaka': 1999}

In [57]:
my_dict['Jimmy Hollywood']

1994

In [45]:
movies.YearOfRelease = movies.YearOfRelease.fillna(movies.Title.map(my_dict))

In [46]:
movies.YearOfRelease = movies.YearOfRelease.astype(int)

In [60]:
movies.sample(10)

,MovieID,YearOfRelease,Title
10423,10424,1993,Carlito's Way
4719,4720,2003,The Work of Director Chris Cunningham
15019,15020,2002,Blue Vinyl
12175,12176,1999,The Fall and Rise of Sherlock Holmes
5622,5623,2003,Daredevil: Bonus Material
14096,14097,2003,MTV: The Real World: Hook-Ups
8606,8607,2003,Shakira: Live and Off the Record
14833,14834,1958,Leonard Bernstein's Young People's Concerts
13797,13798,1920,The Last of the Mohicans
1406,1407,1921,Orphans of the Storm


In [61]:
len(movies.MovieID.unique()), len(ratings.MovieID.unique())

(17770, 92)

In [47]:
complete = pd.merge(ratings, movies)

In [63]:
len(complete.MovieID.unique())

92

In [64]:
complete.sample(20)

,MovieID,UserID,Rating,YearOfRelease,Title
76325,3418,1482158,5.0,1993,Sister Act 2: Back in the Habit
204766,15582,1592816,4.0,2002,Sweet Home Alabama
64119,3253,2393933,3.0,1998,The Opposite of Sex
189786,15582,2155100,3.0,2002,Sweet Home Alabama
139976,11688,2036838,5.0,1957,Heaven Knows
193851,15582,204568,3.0,2002,Sweet Home Alabama
72456,3418,2522960,5.0,1993,Sister Act 2: Back in the Habit
15192,305,156782,5.0,1996,Jack
151307,13927,1342407,4.0,2002,Door to Door
142070,12292,862592,4.0,1993,The Secret of Roan Inish


In [54]:
cond = [complete['MovieID'] == 15785] & [complete['UserID'] == 2136494]

complete.loc[cond]

TypeError: unsupported operand type(s) for &: 'list' and 'list'

In [65]:
check_null(ratings)

MovieID    0
UserID     0
Rating     0
dtype: int64

In [17]:
ratings2 = ratings.pivot(index = 'UserID', columns = 'MovieID', values = 'Rating').fillna(0)

In [84]:
ratings2.head(8)

MovieID,28,48,305,577,595,816,907,992,1061,1744,...,15661,15714,15726,15785,15992,16082,16576,16820,16933,17137
UserID,,,,,,,,,,,,,,,,,,,,,
7,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0
79,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,...,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0
199,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,...,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0
481,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0
769,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
906,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1310,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0
1333,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,...,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0


In [98]:
# from scipy.sparse import csr_matrix

In [100]:
# sparse_mat = csr_matrix(ratings2.values)

In [4]:
import surprise

In [5]:
reader = surprise.Reader(line_format='item user rating', sep=',')

In [6]:
data = surprise.Dataset.load_from_file(filepath, reader=reader)

In [7]:
alg = surprise.SVDpp()

In [13]:
alg.fit(data.build_full_trainset())

In [27]:
import math

In [8]:
alg.predict(iid = '202607', uid = '1744').est

AttributeError: 'SVDpp' object has no attribute 'trainset'

In [10]:
cond1 = ratings.MovieID == 1744
cond2 = ratings.UserID == 202607

In [11]:
ratings.loc[cond1 & cond2]

,MovieID,UserID,Rating
36082,1744,202607,1.0


In [12]:
round(2.50)

2

In [16]:
alg2 = surprise.CoClustering()

In [17]:
alg2.fit(data.build_full_trainset())

In [20]:
alg2.predict(iid = '202607', uid = '1744').est

3.428185079004096

In [24]:
alg3 = surprise.KNNWithMeans(k=41)

In [ ]:
alg3.fit(data.build_full_trainset())

In [16]:
from sklearn.decomposition import TruncatedSVD

In [18]:
svd = TruncatedSVD(n_components = 1)

In [19]:
svd.fit(ratings2)

TruncatedSVD(algorithm='randomized', n_components=1, n_iter=5,
             random_state=None, tol=0.0)

In [20]:
svd.components_

array([[2.71239616e-01, 3.37315141e-02, 1.37639629e-01, 3.00826331e-02,
        6.96438168e-03, 5.81869758e-04, 3.98605791e-02, 6.70357712e-02,
        1.09615490e-02, 4.14160832e-01, 2.11765409e-01, 5.65564164e-04,
        2.76007730e-03, 1.75800628e-03, 5.55835015e-04, 1.52371968e-02,
        9.40657185e-02, 7.65826949e-04, 8.30600611e-04, 2.35413450e-01,
        6.78279243e-02, 7.96697612e-03, 3.78436188e-02, 1.37266301e-03,
        8.35007386e-04, 1.69222199e-03, 9.22065443e-04, 2.03598183e-01,
        1.21381629e-02, 1.44367710e-03, 1.14718275e-03, 8.85412864e-03,
        3.12168088e-02, 1.13447821e-03, 3.65367065e-01, 6.86277345e-03,
        7.54002995e-02, 6.75130254e-04, 3.21695802e-02, 1.53195508e-03,
        8.22614601e-04, 2.22943395e-03, 7.58980891e-04, 3.26403726e-02,
        5.40886992e-02, 8.08452960e-03, 3.04765005e-03, 9.65123391e-03,
        7.37182114e-02, 4.93189807e-04, 8.99220746e-03, 8.13899765e-04,
        1.00594448e-03, 1.11382376e-03, 2.34564780e-02, 1.625598

In [21]:
svd.predict()

AttributeError: 'TruncatedSVD' object has no attribute 'predict'

In [60]:
X = ratings.drop(columns = 'Rating')
y = ratings.Rating

In [59]:
from sklearn.neighbors import KNeighborsClassifier

In [26]:
model = KNeighborsClassifier()

In [27]:
model.fit(X,y)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [33]:
model.predict([
[15582, 2155100]
])

array([3.])

In [62]:
from sklearn.model_selection import cross_validate, ShuffleSplit

In [72]:
for k in range(1,50):
    result = cross_validate(KNeighborsClassifier(n_neighbors = k), X, y, cv = ShuffleSplit(n_splits = 50))
    print(k, result['test_score'].mean())

1 0.2905487751339011
2 0.28039775221705154
3 0.27601984370884186
4 0.2995258582843094
5 0.3144920537360611
6 0.3166643252260954
7 0.3219334445517605
8 0.3271446132232857
9 0.3315356923347089
10 0.33358767231539205
11 0.3364158398454649
12 0.3389349372201246
13 0.340835016243744
14 0.34369303714110105
15 0.3448423917815435
16 0.3462560365264729
17 0.3478461673544648
18 0.34741329352884365
19 0.34992185442093254
20 0.35105803845816147
21 0.35127754851172177
22 0.3519712002809729
23 0.3530652383879182
24 0.3542558609184301
25 0.3550882430415313
26 0.35441390815699364
27 0.35514268153481426
28 0.35633769426639744
29 0.35741680568970063
30 0.35617701290719117
31 0.35662656949688293
32 0.35676529985073313
33 0.35700237070857843
34 0.3584309421371498
35 0.35764509614540346
36 0.3585398191237158
37 0.35881025550970236
38 0.3589612784265519
39 0.3596716129598736
40 0.36024058301870227
41 0.3601931688471332
42 0.3595636140135218
43 0.3598138554745807
44 0.36018702256563356
45 0.3601879006058478


In [66]:
import sklearn

In [67]:
sorted(sklearn.metrics.SCORERS.keys())

['accuracy',
 'adjusted_mutual_info_score',
 'adjusted_rand_score',
 'average_precision',
 'balanced_accuracy',
 'brier_score_loss',
 'completeness_score',
 'explained_variance',
 'f1',
 'f1_macro',
 'f1_micro',
 'f1_samples',
 'f1_weighted',
 'fowlkes_mallows_score',
 'homogeneity_score',
 'jaccard',
 'jaccard_macro',
 'jaccard_micro',
 'jaccard_samples',
 'jaccard_weighted',
 'max_error',
 'mutual_info_score',
 'neg_log_loss',
 'neg_mean_absolute_error',
 'neg_mean_squared_error',
 'neg_mean_squared_log_error',
 'neg_median_absolute_error',
 'normalized_mutual_info_score',
 'precision',
 'precision_macro',
 'precision_micro',
 'precision_samples',
 'precision_weighted',
 'r2',
 'recall',
 'recall_macro',
 'recall_micro',
 'recall_samples',
 'recall_weighted',
 'roc_auc',
 'v_measure_score']

In [70]:
result = cross_validate(KNeighborsClassifier(n_neighbors = 5), X, y, cv = ShuffleSplit(n_splits = 50), scoring = 'accuracy')

In [71]:
result

{'fit_time': array([0.3298471 , 0.31258225, 0.39066386, 0.34769392, 0.27417493,
        0.36846113, 0.3312459 , 0.33522177, 0.29969192, 0.30326915,
        0.30570102, 0.33894086, 0.30427384, 0.29590893, 0.25264096,
        0.32652211, 0.26898098, 0.34637499, 0.27811289, 0.34294796,
        0.29192376, 0.317909  , 0.65603805, 0.7681973 , 0.37565899,
        0.71208692, 0.29312491, 0.262995  , 0.37165022, 0.28681111,
        0.27589488, 0.27367783, 0.27441406, 0.32784796, 0.34334016,
        0.31668615, 0.35512495, 0.24490428, 0.27758408, 0.27634597,
        0.23927999, 0.75293803, 0.30461597, 0.93515015, 0.58625913,
        0.37392116, 0.28459787, 0.32419014, 0.28401399, 0.29986405]),
 'score_time': array([0.93300605, 0.89949703, 0.96154094, 1.05211902, 0.88699698,
        0.86634207, 0.90083289, 0.83103204, 0.82438803, 0.82265711,
        0.9348309 , 0.88665199, 0.82301593, 0.81160593, 0.81518388,
        0.80009079, 0.8190999 , 0.82613301, 0.84302282, 0.83394504,
        0.90330195, 